## Setup

To make the creation of strategies as simple as possible, WDSS has created a Python module which handles gameplay and provides a base strategy for you to work from. The game and base strategy are imported in the following code block.

In [1]:
import os
import sys

if 'google.colab' in str(get_ipython()):
    if not os.path.exists('game-theory-beyond-the-book'):
        !git clone https://github.com/warwickdatascience/game-theory-beyond-the-book.git
    else:
        !git -C game-theory-beyond-the-book pull
    sys.path.append('game-theory-beyond-the-book/src') 
else:
    sys.path.append('../src')
    
from gtbtb import Game, BaseStrategy

If you want to use any other packages/modules, you can import them here. For example, we will use the `random` module for a random strategy. Make sure you only include packages/modules that are part of the standard Python library.

In [2]:
import random

## Example Strategies

### Random Honest

By itself, the iterated game of split-or-steal is not incredibly interesting. When no communication between players is allowed, it is often possible to easily find the optimal strategy of a pair using mathematics directly. If that is so, why are we bothering with simulation? The answer is, so we can take it further than this basic ruleset.

To do this, we allow partipicants to write two additional components to their strategies: intentions and responses. We will begin with the former. This allows a player to broadcast what move they plan to make to the opponent before they commit. This allows the other player to alter their strategy in light of this information. That said, there is nothing stopping a player from lying about what they intend to do, a extra layer of interest we will explore in another notebook.

To broadcast intentions, we must augment our standard strategy definition by adding a `state_intention` method. This should return either `0` or `1` as in `state_action`. For example, we can create a strategy that plays randomly but is at least honest about it. To do this, we can create an _attribute_ of the strategy `self.intention` to store the intention generated in `state_intention` to be accessed in `state_action`.

In [3]:
class RandomHonest(BaseStrategy):
    
    def state_action(self):
        return self.intention
    
    def state_intention(self):
        self.intention = random.choice((0, 1))
        return self.intention

Note, that if we don't define a `state_intention` method, then the default behaviour taken from `BaseStrategy` is to return `None`. 

Let's create a completely trusting strategy that mimics its opponents intentions. To access this intention, we use `self.game.get_history` as in the state management notebook, but replace `"action"` with `"intention"`.

In [4]:
class Trusting(BaseStrategy):
    
    def state_action(self):
        op_int = self.game.get_history(self.op_id, "intention")[-1]
        if op_int is None:
            # Play randomly if no intention given
            return random.choice((0, 1))
        return op_int

Playing these strategies against each other we can see that their moves always line up, both splitting when `RandomHonest` splits and both stealing otherwise, leaving final scores centred around 25.

In [15]:
Game(strategies=(RandomHonest, Trusting), iterations=1000).get_results()

{0: 24.45, 1: 24.45}

### Empty Threatener

Our second extension to the standard iterated split-or-steal game is to allow players to threaten how they will respond to their opponent's next action. We do this by creating a `state_response` method (else `None` is returned by default). This, however, does not need to simply return the value `0` or `1`, but rather a dictionary with keys `0` and `1` with values chosen from `0` and `1`. These will represent what response will be made given each possible move of the opponent. That said, as before, lying is allowed and may create interesting strategies.

For example, we can create a player that threatens to steal next move if their opponent steals but then doesn't follow through.

In [16]:
class EmptyThreatener(BaseStrategy):
    
    def state_action(self):
        return 0
    
    def state_intention(self):
        return self.game.get_history(self.op_id, "action")[-1]
    
    def state_response(self):
        return {0: 0, 1: 1}

Just as with actions and intentions, we can access the opponent's 